In [7]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import numpy as np

In [13]:
# MNISTデータセットのロード
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# データの前処理
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255

rate=0.2  #ココで追加データに付与するノイズを調整する
noise = rate * np.random.rand(*X_train.shape)
Xtrain_dirt = X_train + noise

X_train = np.concatenate((X_train, Xtrain_dirt))
y_train = np.concatenate((y_train, y_train))

# ラベルのone-hotエンコーディング
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [14]:
# モデルの構築
#i=0.1
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(6, 6), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
#model.add(Dense.Dropout(i))

In [15]:
# モデルのコンパイル
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# モデルの訓練
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

# モデルの評価
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 14, 14, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 476,938 (1.82 MB)

 Trainable params: 476,938 (1.82 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 70s 18ms/step - accuracy: 0.9451 - loss: 0.1721 - val_accuracy: 0.9899 - val_loss: 0.0301
Epoch 2/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 66s 18ms/step - accuracy: 0.9926 - loss: 0.0236 - val_accuracy: 0.9889 - val_loss: 0.0336
Epoch 3/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 66s 18ms/step - accuracy: 0.9961 - loss: 0.0117 - val_accuracy: 0.9927 - val_loss: 0.0260
Epoch 4/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 67s 18ms/step - accuracy: 0.9976 - loss: 0.0074 - val_accuracy: 0.9924 - val_loss: 0.0296
Epoch 5/5
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 66s 18ms/step - accuracy: 0.9980 - loss: 0.0060 - val_accuracy: 0.9916 - val_loss: 0.0406
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9900 - loss: 0.0503
Test accuracy: 0.9916


In [17]:
model.save('mnist_model.keras')

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('mnist_model.keras')
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(), loss='mse')

In [ ]:
image_path = '/Users/fukudakyoutarou/Desktop/IMG_6332.HEIC'  # 手書き文字が含まれる画像のパス
image = cv2.imread(image_path)
if image is None:
    print("画像の読み込みに失敗しました。ファイルパスを確認してください。")
else:
    # 色空間変換を行う
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 二値化処理
_, binary_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY_INV)

# 輪郭を検出
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 結果を可視化するために元の画像をカラーで読み込み
output_image = cv2.cvtColor(gray_image, cv2.COLOR_GRAY2BGR)  # グレースケールをカラー画像に変換
cv2.drawContours(output_image, contours, -1, (0, 255, 0), 3)  # 輪郭を描画（緑色）

In [ ]:

# 各輪郭を処理
for contour in contours:
    # 外接矩形を取得
    x, y, w, h = cv2.boundingRect(contour)
    
    # 小さすぎる領域を除外
    if w > 10 and h > 10:
        # 対象領域を切り出し
        digit = binary_image[y:y+h, x:x+w]
        
        # MNISTフォーマットにリサイズ（28x28）
        resized_digit = cv2.resize(digit, (28, 28), interpolation=cv2.INTER_AREA)
        
        # ピクセル値を正規化（0～1）
        normalized_digit = resized_digit / 255.0
        
        # モデルの入力形式に変換
        input_digit = normalized_digit.reshape(1, 28, 28, 1)
        
        # 学習済みモデルで予測
        prediction = model.predict(input_digit)
        predicted_label = np.argmax(prediction)
        
        # 結果を描画
        cv2.rectangle(output_image, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(output_image, str(predicted_label), (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

In [ ]:
cv2.imshow('Detected Digits', output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()